# BLUE SQUARES

In [7]:
# imports
import cv2
import numpy as np
import os
import math

## Download

In [8]:
image_path = "./archive/images"

## Pre-Process


In [9]:
def smooth(img): 

    # remove noise
    # Using a Median Filter
    img_smoothed = cv2.medianBlur(img, 5)

    return img_smoothed

def contrastAdjust(img):

    # adjust contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    # convert to YCrCb and apply CLAHE on Y component
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    # apply CLAHE on component Y
    img_YCrCb[:, :, 0] = clahe.apply(img_YCrCb[:, :, 0])

    img_clean = cv2.cvtColor(img_YCrCb, cv2.COLOR_YCrCb2BGR)
    
    return img_clean

In [10]:
# TODO: CHANGE THIS

def calcAngles(cnt_img, corners):
    angles = []
    
    # Calculate euclidean distance between each corner
    d1 = int(math.sqrt(math.pow(corners[0] - corners[2], 2) +
                       math.pow(corners[1] - corners[3], 2)))
    d2 = int(math.sqrt(math.pow(corners[2] - corners[4], 2) +
                       math.pow(corners[3] - corners[5], 2)))
    d3 = int(math.sqrt(math.pow(corners[4] - corners[6], 2) +
                   math.pow(corners[5] - corners[7], 2)))
    d4 = int(math.sqrt(math.pow(corners[6] - corners[0], 2) +
                   math.pow(corners[7] - corners[1], 2)))

    max_radius = min([d1, d2, d3, d4])

    for k in range(len(corners) - 1):
        if(k % 2 != 0):
            continue

        blank_img = np.zeros((len(cnt_img), len(cnt_img[0])), np.uint8)
        cv2.circle(blank_img, (corners[k], corners[k + 1]),
                  max_radius // 2, (255, 255, 255))
        intersect_img = cv2.bitwise_and(cnt_img, blank_img)

        intersect_pts = np.where(intersect_img > 1)

        if(len(intersect_pts[0]) < 2 or len(intersect_pts[1]) < 2):
            angles.append(0)
            continue

        vector1 = (intersect_pts[1][0] - corners[k],
                   intersect_pts[0][0] - corners[k + 1])
        vector2 = (intersect_pts[1][1] - corners[k],
                   intersect_pts[0][1] - corners[k + 1])

        scalar_p = vector1[0] * vector2[0] + vector1[1] * vector2[1]
        norm1 = math.sqrt(math.pow(vector1[0], 2) + math.pow(vector1[1], 2))
        norm2 = math.sqrt(math.pow(vector2[0], 2) + math.pow(vector2[1], 2))

        angle = math.acos(scalar_p / (norm1 * norm2)) * 180 / math.pi
        angles.append(angle)

    return angles


## Feature Recognition 

In [44]:
#features: blue rectangle

def featureRecognition(img): 

    # blue segmentation
    # Converts images from BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Bounds adjusted to identify shades of blue correspondent to traffic signs
    lower_blue = np.array([105, 90, 90])
    upper_blue = np.array([150, 255, 255])

    # Here we are defining range of bluecolor in HSV
    # This creates a mask of blue coloured
    # objects found in the frame.
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # The bitwise and of the frame and mask is done so
    # that only the blue coloured objects are highlighted
    # and stored in res
    res = cv2.bitwise_and(img, img, mask=mask)
    cv2.imshow('original', img)
    cv2.imshow('mask', mask)
    cv2.imshow('res', res)



    # TEST SEGMENTATION
    cv2.waitKey()

    # Destroys all of the HighGUI windows.
    cv2.destroyAllWindows()


    # Post-Segmentation smoothing
    res = smooth(res)  


    # Contours on Resulting Image
    gray_img = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    # Apply Canny
    canny = cv2.Canny(gray_img, 130, 255, 1)
    
    # Find Contours
    # RETR_CCOMP: retrieves all of the contours and organizes them into a two-level hierarchy. At the top level, there are external boundaries of the components. 
    # At the second level, there are boundaries of the holes. If there is another contour inside a hole of a connected component, it is still put at the top level.
    cnts = cv2.findContours(
        canny, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]


    for c in range(len(cnts)):
        cnt_len = cv2.arcLength(cnts[c], True)

        # FIXME: WHY??
        if(cnt_len <= 70):
            cv2.drawContours(img, [cnts[c]], 0, (0, 255, 0), 3)
            continue 


        #  recognize most approximate Poly-shape to contour to draw
        approx_poly = cv2.approxPolyDP(
            cnts[c], 0.03 * cv2.arcLength(cnts[c], True), True)


        # # FIXME: WHY?
        # # calculate image area
        # area = img.shape[0]*img.shape[1]
        # # calculate ratio between image area and poly area
        # ratio = int(cv2.contourArea(approx_poly)/float(area) * 100000.0)

        # if ratio <= 65:
        #     cv2.drawContours(img, [cnts[c]], 0, (0, 255, 0), 3)
        #     continue

        # if(len(approx_poly) <= 4):
        #     cv2.drawContours(img, [cnts[c]], 0, (0, 255, 0), 3)



        # rectangle recognition
        ravel = approx_poly.ravel()
        n_sides = len(approx_poly)
        angle_cdt = False 

        # Nº of Sides is 4 = Square/Rectangle
        if(n_sides == 4): 
            angle_cdt = True 

            contour_only_img = np.zeros((len(img), len(img[0])), np.uint8)
            angles = calcAngles(cv2.drawContours(contour_only_img, [approx_poly], -1, (255), 1), ravel)

            for i in range(len(angles)):
                if(abs(angles[i] - 90) > 10):
                    angle_cdt=False
                    break
            
        if(angle_cdt):
            img = cv2.drawContours(img, [approx_poly], -1, (0, 255, 255), 3)

    
    cv2.imshow("result", img)



    # TEST contours
    cv2.waitKey()

    # Destroys all of the HighGUI windows.
    cv2.destroyAllWindows()


    


## Main Loop

In [48]:
img = cv2.imread(os.path.join(image_path, 'road318.png'))
img_clean = contrastAdjust(smooth(img))
featureRecognition(img_clean)

## Results